In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os

In [5]:
def load_file(file_path):
    try:
        with open(file_path) as f_in:
            lines = f_in.readlines()
        return lines
    except IOError as err:
        print('Can not open file: ' + file_path)
        return 'nan'

def parse_pssm(filelines, winsize=51, pssm_root=None, mutated_pos=None, most1024=False):
#     pssm_root = '/lustre/home/acct-bmelgn/bmelgn-2/QianWei/app/psipred_file/psipred/BLAST+/v20200727/pssm'
    filelines = load_file(os.path.join(pssm_root, filelines + '.pssm'))
    if filelines == 'nan':
        return 'nan'
    pssmvalue = np.array([])
    for line in filelines:
        if len(line.split()) == 44:
            pssmvalue = np.r_[pssmvalue, np.array(line.split()[2:22]).astype(float)]
    pssmvalue = np.reshape(pssmvalue, (-1, 20))
    if pssmvalue.shape[0] < 1024:
        pssmvalue = np.r_[pssmvalue, np.zeros([1024 - pssmvalue.shape[0], 20])]
    if most1024:
        if pssmvalue.shape[0] > 1024:
            pssmvalue = pssmvalue[:1024, :]
    if mutated_pos != None:
        pssmvalue = np.r_[np.zeros([25, 20]), pssmvalue, np.zeros([25, 20])]
        pssmvalue = pssmvalue[mutated_pos - 1: mutated_pos + 50, :]
    
    return pssmvalue

In [2]:
df = pd.read_csv('../data/skempi_v2.csv', delimiter=';')

In [3]:
df.head()

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,koff_mut_parsed,koff_wt (s^(-1)),koff_wt_parsed,dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IASP,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   #Pdb                          7085 non-null   object 
 1   Mutation(s)_PDB               7085 non-null   object 
 2   Mutation(s)_cleaned           7085 non-null   object 
 3   iMutation_Location(s)         7085 non-null   object 
 4   Hold_out_type                 3311 non-null   object 
 5   Hold_out_proteins             7085 non-null   object 
 6   Affinity_mut (M)              7085 non-null   object 
 7   Affinity_mut_parsed           6800 non-null   float64
 8   Affinity_wt (M)               7085 non-null   object 
 9   Affinity_wt_parsed            7083 non-null   float64
 10  Reference                     7085 non-null   object 
 11  Protein 1                     7085 non-null   object 
 12  Protein 2                     7085 non-null   object 
 13  Tem

In [5]:
# first delete na rows
print(df.shape)
df = df.dropna(subset=['Affinity_mut_parsed'])
df = df.dropna(subset=['Affinity_wt_parsed'])
df = df.dropna(subset=['Temperature'])
print(df.shape)

# # drop items with insertion code
df['check'] = [all(xx[2:-1].isnumeric() for xx in x.split(',')) for x in df['Mutation(s)_PDB']]
df = df[df['check'] == True]
df = df.drop(['check'], axis=1)
print(df.shape)

(7085, 29)
(6794, 29)
(6729, 29)


In [6]:
df['Temperature'] = df['Temperature'].str.replace('\(assumed\)', '')
df['Temperature'] = pd.to_numeric(df['Temperature'])

In [7]:
# from skempi2 website,  ΔG = (8.314/4184)*(273.15 + 25.0) * ln(wt)
ddg = (8.314/4184) * df['Temperature'].values * np.log(df['Affinity_mut_parsed'].values / df['Affinity_wt_parsed'].values)
df['ddg'] = ddg

In [ ]:
df['PDB'], df['chain1'], df['chain2'] = df['#Pdb'].str.split('_', n=2).str
# delete items with more than 2 chains in a protein-protein interaction
df = df[df['chain1'].str.len() == 1]
df = df[df['chain2'].str.len() == 1]

In [9]:
df.head()

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes,Method,SKEMPI version,ddg,PDB,chain1,chain2
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,IASP,1,2.248833,1CSE,E,I
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,IASP,1,1.172229,1CSE,E,I
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,IASP,1,6.671276,1CSE,E,I
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,IASP,1,2.940988,1CSE,E,I
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,NaN,NaN,NaN,NaN,IASP,1,4.350434,1CSE,E,I


In [10]:
# get position(s) for each item
df['refaa'] = [[xx[0] for xx in x.split(',')] for x in df['Mutation(s)_PDB']]
df['mutaa'] = [[xx[-1] for xx in x.split(',')] for x in df['Mutation(s)_PDB']]
df['mutchain'] = [[xx[1] for xx in x.split(',')] for x in df['Mutation(s)_PDB']]
df['pos0'] = [[int(xx[2:-1]) for xx in x.split(',')] for x in df['Mutation(s)_PDB']]
df['pos1'] = [[int(xx[2:-1]) for xx in x.split(',')] for x in df['Mutation(s)_cleaned']]
print(df.shape)

(4158, 38)


In [11]:
# filter items with mutation in both protein
df = df[[len(set(x)) == 1 for x in df['mutchain']]]
df['mutchain_s'] = [x[0] for x in df['mutchain']]
df.shape

(3681, 39)

In [12]:
parchain = []
for i in df.index:
    if df.loc[i, 'chain1'] == df.loc[i, 'mutchain_s']:
        parchain.append(df.loc[i, 'chain2'])
    elif df.loc[i, 'chain2'] == df.loc[i, 'mutchain_s']:
        parchain.append(df.loc[i, 'chain1'])
    else:
        print('no match chain for pdb {}'.format(df.loc[i, 'PDB']))
df['parchain_s'] = parchain

## generate PDB IDs for retrieve sequences from PDB

In [13]:
# a = df['PDB'].copy()
# a = a.drop_duplicates(keep='first')
# a.to_csv('../data/all_skempi2.csv', index=False, header=False)

In [14]:
# # got PDB FASTA files as 
# pdbid = []
# chain = []
# seq = []
# tmpid = ''
# tmpch = ''
# with open('../data/test_sets/skempi2/rcsb_pdb_skempi2.fasta', 'r') as f:
#     for line in f:
#         line = line.strip()
#         if line[0] == '>':
#             line = line.split('|')
#             tmpid = line[0].split('_')[0][1:]
#             if 'Chains' in line[1]:
#                 tmpch = line[1][7:].split(',')
#             else:
#                 tmpch = line[1][6:]
#         else:
#             for ch in tmpch:
#                 pdbid.append(tmpid)
#                 chain.append(ch)
#                 seq.append(line)
# df_seq = pd.DataFrame({'PDB': pdbid, 'mutchain_s': chain, 'seq': seq})
            
# #             for ch in tmpch:
# #                 pdbid.append()

In [17]:
# mut_seq = []
# # seq51 = []
# # mut_seq51 = []
# tmp_seq = ''
# # tmp_seq51 = ''
# # tmp_mut_seq = ''
# for i in df_all.index:
#     tmp_seq = df_all.loc[i, 'seq']
#     pos0 = df_all.loc[i, 'pos0']
#     pos1 = df_all.loc[i, 'pos1']
#     ori = df_all.loc[i, 'refaa']
#     mut = df_all.loc[i, 'mutaa']
# #     if (tmp_seq[pos0[0] - 1] == ori[0]) and (tmp_seq[pos1[0] - 1] == ori[0]):
# #         print(i)
#     if tmp_seq[pos1[0] - 1] == ori[0]:
#         for j, _ in enumerate(pos1):
#             if tmp_seq[pos1[j] - 1] == ori[j]:
#                 tmp_seq = tmp_seq[0:(pos1[j] - 1)] + mut[j] + tmp_seq[pos1[j]:]
#             else:
#                 print('0true, {}false, {}row'.format(df_all.loc[i, 'PDB'], i))
#     elif tmp_seq[pos0[0] - 1] == ori[0]:
#         for j, _ in enumerate(pos0):
#             if tmp_seq[pos0[j] - 1] == ori[j]:
#                 tmp_seq = tmp_seq[0:(pos0[j] - 1)] + mut[j] + tmp_seq[pos0[j]:]
#             else:
#                 print('0true, {}false'.format(j))
#     else:
#         print('no match {}, {} row'.format(df_all.loc[i, 'PDB'], i))
#     mut_seq.append(tmp_seq)

## encode PDB files

In [18]:
aadict = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C',
          'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I',
          'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P',
          'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'}

In [19]:
mut_seq = []
ori_seq = []
par_seq = []
for i in df.index:
#     tmp_seq = df.loc[i, 'seq']
#     pos0 = df.loc[i, 'pos0']
    pos1 = df.loc[i, 'pos1']
    ori = df.loc[i, 'refaa']
    mut = df.loc[i, 'mutaa']
    pdbid = df.loc[i, 'PDB']
    mutchain = df.loc[i, 'mutchain_s']
    parchain = df.loc[i, 'parchain_s']
    tmp_seq = ''
    tmp_par = ''
    with open('../data/test_sets/skempi2/PDBs/' + pdbid + '.mapping', 'r') as f:
        for line in f:
            line = line.strip()
            line = ' '.join(line.split())
            line = line.split(' ')
            if line[1][0] == mutchain:
                try:
                    tmp_seq += aadict[line[0]]
                except:
                    tmp_seq = 'falseseqaa'
                    tmp_par = 'falseseqaa'
#                     print('error seq pdbid {}, illegal aa {}'.format(pdbid, line[0]))
                    break
            if line[1][0] == parchain:
                try:
                    tmp_par += aadict[line[0]]
                except:
                    # use tmp seq again to simplify following steps
                    tmp_par = 'falseseqaa'
                    tmp_seq = 'falseseqaa'
#                     print('error seq pdbid {}, illegal aa {}'.format(pdbid, line[0]))
                    break

    if tmp_seq == 'falseseqaa':
        ori_seq.append(tmp_seq)
        mut_seq.append(tmp_seq)
        par_seq.append(tmp_par)
        continue
    else:
        ori_seq.append(tmp_seq)
        par_seq.append(tmp_par)
    for j in range(len(pos1)):
        try:      
            if tmp_seq[pos1[j] - 1] != ori[j]:
                print('false match, row {}, pdbid: {}'.format(i, pdbid))
            tmp_seq = tmp_seq[0:(pos1[j] - 1)] + mut[j] + tmp_seq[pos1[j]:]
        except:
            print('out of range, row {}, pdbid: {}, pos {}, tmp_seq len {}'.format(i, pdbid, pos1[j], len(tmp_seq)))
    mut_seq.append(tmp_seq)

In [20]:
df['ori_seq'], df['mut_seq'], df['par_seq'] = ori_seq, mut_seq, par_seq

In [21]:
df = df[~(df['mut_seq'] == 'falseseqaa')]

In [22]:
df.head()

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,refaa,mutaa,mutchain,pos0,pos1,mutchain_s,parchain_s,ori_seq,mut_seq,par_seq
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,[L],[G],[I],[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,[L],[S],[I],[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,[L],[P],[I],[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,[L],[I],[I],[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTIDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,[L],[D],[I],[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...


In [23]:
df_w = df[df['pos1'].str.len() == 1]
df_w = df_w.reset_index(drop=True)

In [24]:
pos1_w = [x[0] for x in df_w['pos1']]
df_w['pos1_w'] = pos1_w

In [25]:
ori_win = []
mut_win = []
tmp = ''
for i in df_w.index:
    tmp = '0' * 25 + df_w.loc[i, 'ori_seq'] + '0' * 25
    ori_win.append(tmp[df_w.loc[i, 'pos1_w'] - 1: df_w.loc[i, 'pos1_w'] + 50])
    tmp = '0' * 25 + df_w.loc[i, 'mut_seq'] + '0' * 25
    mut_win.append(tmp[df_w.loc[i, 'pos1_w'] - 1: df_w.loc[i, 'pos1_w'] + 50])

df_w['ori_win'], df_w['mut_win'] = ori_win, mut_win

In [26]:
df_w.head()

,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,iMutation_Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_mut_parsed,Affinity_wt (M),Affinity_wt_parsed,...,pos0,pos1,mutchain_s,parchain_s,ori_seq,mut_seq,par_seq,pos1_w,ori_win,mut_win
0,1CSE_E_I,LI45G,LI38G,COR,Pr/PI,Pr/PI,5.26E-11,5.260000e-11,1.12E-12,1.120000e-12,...,[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,38,QAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNH...,QAREYFTLHYPQYNVYFLPEGSPVTGDLRYNRVRVFYNPGTNVVNH...
1,1CSE_E_I,LI45S,LI38S,COR,Pr/PI,Pr/PI,8.33E-12,8.330000e-12,1.12E-12,1.120000e-12,...,[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,38,QAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNH...,QAREYFTLHYPQYNVYFLPEGSPVTSDLRYNRVRVFYNPGTNVVNH...
2,1CSE_E_I,LI45P,LI38P,COR,Pr/PI,Pr/PI,1.02E-07,1.020000e-07,1.12E-12,1.120000e-12,...,[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,38,QAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNH...,QAREYFTLHYPQYNVYFLPEGSPVTPDLRYNRVRVFYNPGTNVVNH...
3,1CSE_E_I,LI45I,LI38I,COR,Pr/PI,Pr/PI,1.72E-10,1.720000e-10,1.12E-12,1.120000e-12,...,[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTIDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,38,QAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNH...,QAREYFTLHYPQYNVYFLPEGSPVTIDLRYNRVRVFYNPGTNVVNH...
4,1CSE_E_I,LI45D,LI38D,COR,Pr/PI,Pr/PI,1.92E-09,1.920000e-09,1.12E-12,1.120000e-12,...,[45],[38],I,E,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVR...,KSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDDLRYNRVR...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,38,QAREYFTLHYPQYNVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNH...,QAREYFTLHYPQYNVYFLPEGSPVTDDLRYNRVRVFYNPGTNVVNH...


In [32]:
# df.to_pickle('../data/test_sets/skempi2/skempi2_wholelength.dataset')
df_w.to_pickle('../data/test_sets/skempi2/skempi2_window.dataset')

## reload for pssm calculation

In [2]:
# df = pd.read_pickle('../data/test_sets/skempi2/skempi2_wholelength.dataset')
df_w = pd.read_pickle('../data/test_sets/skempi2/skempi2_window.dataset')

In [15]:
# df_w[df_w['#Pdb'] == '3Q8D_A_E']
df_w.shape

(3000, 46)

In [16]:
df_w['mutAC'] = df_w['#Pdb'] + '_' + df_w['Mutation(s)_cleaned'].str.replace(',', '_')
df_w['oriAC'] = df_w['#Pdb'] + '_' + df_w['mutchain_s']
df_w['parAC'] = df_w['#Pdb'] + '_' + df_w['parchain_s']

In [ ]:
# change it to your PSSM root
pssm_path = '/lustre/home/acct-bmelgn/bmelgn-2/QianWei/app/psipred_file/psipred/BLAST+/skempi2_20200904/pssm'

In [ ]:
pssm_par0 = [parse_pssm(x, pssm_root=pssm_path) for x in df_w['parAC']]
# pssm 51 window only available to single mutation items
df_w['pssm_par0'] = pssm_par0
df_w = df_w[~(df_w['pssm_par0'] == 'nan')]
print('after pssm_par0: {}'.format(df_w.shape))

pssm_mut0 = [parse_pssm(df_w.loc[i, 'oriAC'], pssm_root=pssm_path, mutated_pos=df_w.loc[i, 'pos1_w']) for i in df_w.index]
# pssm 51 window only available to single mutation items
df_w['pssm_mut0_win'] = pssm_mut0
df_w = df_w[~(df_w['pssm_mut0_win'] == 'nan')]
print('after pssm_mut0: {}'.format(df_w.shape))

pssm_mut1 = [parse_pssm(df_w.loc[i, 'mutAC'], pssm_root=pssm_path, mutated_pos=df_w.loc[i, 'pos1_w']) for i in df_w.index]
# pssm 51 window only available to single mutation items
df_w['pssm_mut1_win'] = pssm_mut1
df_w = df_w[~(df_w['pssm_mut1_win'] == 'nan')]
print('after pssm_mut1: {}'.format(df_w.shape))

In [27]:
df_w.to_pickle('../data/skempi2_window_with_pssm.dataset')